In [ ]:
import pandas
import numpy
import requests
import time

In [ ]:
# Set path --- Link to the box folder with your name
# Download Box Desktop to copy the pathname

# Input
# Folder name: step3-outputfile
box_path_1 = {enterdirectorytofolder}

# Output
# Folder name: step3-inputfile
box_path_2 = {enterdirectorytofolder}

In [ ]:
# Import known retraction list-2
date = pd.read_csv(box_path_1 + '-knownretractionlist-2.csv')
date.info()

In [ ]:
filter_1 = date[date.source.str.contains('Crossref')]
filter_2 = filter_1[~filter_1.source.str.contains('Retraction Watch')]

filter_2.source.value_counts()

In [ ]:
url_base = 'https://api.crossref.org/works/{}'

crossref_dois = filter_2['DOI'].tolist()
crossref_dois_chunks = np.array_split(crossref_dois, 104)


output = box_path_2+ '-retractionyear-crossref.tsv'
with open(output, 'a') as fout:
    line = ['DOI', 'retraction-date']
    line = '\t'.join(line) + '\n'
    fout.write(line)
    
for chunk in crossref_dois_chunks:
    start_time = time.time()
    for doi in chunk:
        try:
            url = url_base.format(doi)
            result = requests.get(url)
            result = result.json()
            #print(result)
            #print(result['message'])

            date_parts = result['message']['update-to'][0]['updated']['date-parts'][0]
            year = date_parts[0]

            new_line = [doi, str(year)]
            new_line = '\t'.join(new_line) + '\n'
            with open(output, 'a') as fout:
                fout.write(new_line)
        except:
            print(doi)

        end_time = time.time()

    process_time = end_time - start_time
    
    time.sleep(3)

print(process_time)